In [1]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

class Load_Preprocess(model,tokenizer):
    def __init__(self):

        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        self.model = AutoModel.from_pretrained(model)
        self.word2index = {}
        self.max_len = 0

        self.Label2num = {'Entailment': 1, "Contradiction": 0}

        self.train_df = self.sentence_similarity(model, tokenizer, '/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/ctakes/Dev_Statements_cTAKES_Processed.json')
        self.val_df = self.sentence_similarity(model, tokenizer, '/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/ctakes/Dev_Statements_cTAKES_Processed.json')
        self.test_df = self.sentence_similarity(model, tokenizer, '/content/drive/MyDrive/SemEval2024/Task-2-SemEval-2024-main/ctakes/Dev_Statements_cTAKES_Processed.json')

    def sentence_similarity(self, model, tokenizer, file_path):

      df = pd.read_csv(file_path)

      inputs1 = tokenizer(df['statement'], return_tensors="pt", truncation=True, padding=True)
      inputs2 = tokenizer(df['ctr_trimmed'], return_tensors="pt", truncation=True, padding=True)

      with torch.no_grad():
          outputs1 = model(**inputs1)
          outputs2 = model(**inputs2)

      embeddings1 = outputs1.pooler_output
      embeddings2 = outputs2.pooler_output

      similarity_score = torch.nn.functional.cosine_similarity(embeddings1, embeddings2).item()

      return similarity_score